In [ ]:
# default_exp model

# Models

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import warnings
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [ ]:
#export
from isic.dataset import SkinDataModule
from isic.layers import LabelSmoothingCrossEntropy
from isic.callback.hyperlogger import HyperparamsLogger
from isic.callback.logtable import LogTableMetricsCallback
from isic.callback.mixup import MixupDict
from isic.callback.cutmix import CutmixDict
from isic.callback.freeze import FreezeCallback, UnfreezeCallback
from isic.utils.core import reduce_loss, generate_val_steps
from isic.utils.model import apply_init, get_bias_batchnorm_params, apply_leaf, print_grad_module, create_body, create_head, lr_find, freeze, unfreeze

In [ ]:
#export
class Model(LightningModule):
    def __init__(self, steps_epoch, epochs=30, lr=1e-2, wd=0., n_out=7, concat_pool=True, arch='resnet50'):
        super().__init__()
        self.save_hyperparameters()
        # create body
        body, self.split, num_ftrs = create_body(arch)
        # create head
        head = create_head(num_ftrs, n_out)
        
        #model
        self.model = nn.Sequential(body, head)
        apply_init(self.model[1])
        
        # Setup so that batchnorm will not be freeze
        for p in get_bias_batchnorm_params(self.model):
            p.force_train = True
        
        n_groups = self.create_opt(lr)
        freeze(self.model, n_groups)
        
        self.loss_func = LabelSmoothingCrossEntropy()

    def get_params(self):
        return self.split(self.model)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss)
        result.log('train_acc', acc, prog_bar=True)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True) 
        result.log('val_acc', acc, prog_bar=True)
        return result
    
    def create_opt(self, lr=None):
        if lr is None:
            lr = self.hparams.lr
        param_groups = self.get_params()
        n_groups = len(param_groups)

        def _inner():
            print('override_called')
            
            lrs = generate_val_steps(lr, n_groups)
            assert len(lrs) == n_groups, f"Trying to set {len(lrs)} values for LR but there are {n_groups} parameter groups."
            grps = []
            for i in range(n_groups):
                grps.append({
                    "params": param_groups[i],
                    "lr": lrs[i]
                })
            print(lrs)
            opt = torch.optim.Adam(grps, 
                        lr=1e-2, weight_decay=self.hparams.wd
            )
            scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=lrs, steps_per_epoch=self.hparams.steps_epoch, epochs=self.hparams.epochs)
            sched = {
                'scheduler': scheduler, # The LR schduler
                'interval': 'step', # The unit of the scheduler's step size
                'frequency': 1, # The frequency of the scheduler
                'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
            }
            return [opt], [sched]
        self.configure_optimizers = _inner
        return n_groups

In [ ]:
message_formater = "You have set {0} number of classes if different from predicted {0} and target {0} number of classes"
warnings.filterwarnings("ignore", message_formater.format("(.*)"), category=UserWarning)

In [ ]:
dm = SkinDataModule()
dm.prepare_data()
dm.setup('fit')

In [ ]:
EPOCHS = 10
STEPS_EPOCH = 1

In [ ]:
# init model
model = Model(steps_epoch=STEPS_EPOCH, epochs=EPOCHS, lr=1e-2, arch='resnet18')
n_groups = model.create_opt(slice(1e-3))

In [ ]:
freeze(model, n_groups)

In [ ]:
lr_find(model, dm,lr_find=False,verbose=True)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type                       | Params
---------------------------------------------------------
0 | model     | Sequential                 | 11 M  
1 | loss_func | LabelSmoothingCrossEntropy | 0     


override_called
[0.0001, 0.0001, 0.001]


Saving latest checkpoint..



Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
[False]
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
ReLU(inplace=True)
[]
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
[]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
[False]
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
ReLU(inplace=True)
[]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
[False]
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
[False]
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
ReLU(inplace=True)
[]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
[False]
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [ ]:
hp_log = HyperparamsLogger()

trainer = pl.Trainer(max_epochs=EPOCHS, callbacks=[LogTableMetricsCallback(), hp_log], fast_dev_run=False, limit_val_batches=0, limit_train_batches=0.01)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type                       | Params
---------------------------------------------------------
0 | model     | Sequential                 | 25 M  
1 | loss_func | LabelSmoothingCrossEntropy | 0     


override_called
[0.0001, 0.0001, 0.001]


C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint..


1

In [ ]:
trainer.optimizers[0]

Adam (
Parameter Group 0
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 4e-05
    lr: 0.0005200000000000001
    max_lr: 0.001
    max_momentum: 0.95
    min_lr: 4e-09
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 0.0012649110640673518
    lr: 0.016443843832875574
    max_lr: 0.03162277660168379
    max_momentum: 0.95
    min_lr: 1.2649110640673517e-07
    weight_decay: 0.0

Parameter Group 2
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 0.04
    lr: 0.52
    max_lr: 1.0
    max_momentum: 0.95
    min_lr: 4e-06
    weight_decay: 0.0
)

In [ ]:
# Unfreeze training
hp_log = HyperparamsLogger()
freeze_cb = FreezeCallback()

trainer = pl.Trainer(max_epochs=EPOCHS, callbacks=[LogTableMetricsCallback(), hp_log, unfreeze_cb], fast_dev_run=True, limit_val_batches=0, limit_train_batches=0.01)
model.create_opt(1e-10)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 25 M  
1 | loss_func | CrossEntropyLoss | 0     


override_called
wtf


epoch,train_loss,train_acc,val_loss,val_acc
1,3.256,0.156250,4.279556,0.000000


Saving latest checkpoint..


1

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 4636), started 4 days, 18:33:26 ago. (Use '!kill 4636' to kill it.)

In [ ]:
from nbdev.export import *
notebook2script('model.ipynb')

Converted cb_mixup.ipynb.
